# Data exploration

## Load paths

In [ ]:
def umlaut(string):
    string += ""
    string = string.replace("Ã¼", "ü")
    string = string.replace("Ã¤", "ä")    
    string = string.replace("Ã¶", "ö")    
    string = string.replace("Ãœ", "Ü")    
    string = string.replace("Ã„", "Ä")    
    string = string.replace("Ã–", "Ö")    
    string = string.replace("ÃŸ", "ß")    
    return string

def replace_umlauts(dictionary):
    try:
        dictionary = umlaut(dictionary)
    except TypeError:
        copy = dictionary
        for k, v in dictionary.items():
            copy[k] = replace_umlauts(v)
        dictionary = copy
    return dictionary

In [ ]:
import json

with open('data/paths.json') as f:
    data = json.load(f)
    data = replace_umlauts(data)

data

In [ ]:
keywordFile = data['keywords']['file']
keywordFile

## Load data

In [ ]:
from docx import Document
document = Document(keywordFile)
table = document.tables[0]
cell = table.rows[0].cells[0]
text = cell.text
text